In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
from setuptools.command.rotate import rotate


def get_items(response):
    # 응답 데이터를 텍스트로 변환
    xml_string = response.text  # 또는 response.content.decode('utf-8')
    
    root = ET.fromstring(xml_string)
    item_list = []
    
    # 모든 row 엘리먼트 찾기
    rows = root.findall('row')
    
    # 각 row 엘리먼트에서 데이터 추출
    for row in rows:
        elements = row.findall('*')
        data = {}
        for element in elements:
            tag = element.tag.strip()
            text = element.text.strip() if element.text else None
            data[tag] = text
        item_list.append(data)
    
    return item_list

# items_list = get_items(res)
# items = pd.DataFrame(items_list)
# items.head()

In [2]:
import requests
import datetime
import certifi
import ssl

def get_data(statbl_id, datacycle_cd, cls_id):
    url = "https://www.reb.or.kr/r-one/openapi/SttsApiTblData.do?"
    service_key = "37f7a534514349d48f2cf77dc2691da3"
    

    call_code = "Key=" + service_key + "&" +\
        "pIndex=1&pSize=1000&" +\
        "STATBL_ID=" + statbl_id + "&" +\
        "DTACYCLE_CD=" + datacycle_cd +\
        "&START_WRTTIME=2015&END_WRTTIME=2025" +"&" +\
        "CLS_ID=" + cls_id
    
    print(url+call_code)
    res = requests.get(url + call_code, verify=certifi.where())
    return res

In [10]:
STATBL_ID = "A_2024_00064"
cls_id_list = ["530088", "530089", "530090"]
#cls_id_list = ["500009"]
tx_list = []
for cls_id in cls_id_list:
    res = get_data(STATBL_ID, "MM", cls_id)
    tx_list += get_items(res)
    
len(tx_list)
apt_indi = pd.DataFrame(tx_list)
apt_indi

https://www.reb.or.kr/r-one/openapi/SttsApiTblData.do?Key=37f7a534514349d48f2cf77dc2691da3&pIndex=1&pSize=1000&STATBL_ID=A_2024_00064&DTACYCLE_CD=MM&START_WRTTIME=2015&END_WRTTIME=2025&CLS_ID=530088
https://www.reb.or.kr/r-one/openapi/SttsApiTblData.do?Key=37f7a534514349d48f2cf77dc2691da3&pIndex=1&pSize=1000&STATBL_ID=A_2024_00064&DTACYCLE_CD=MM&START_WRTTIME=2015&END_WRTTIME=2025&CLS_ID=530089
https://www.reb.or.kr/r-one/openapi/SttsApiTblData.do?Key=37f7a534514349d48f2cf77dc2691da3&pIndex=1&pSize=1000&STATBL_ID=A_2024_00064&DTACYCLE_CD=MM&START_WRTTIME=2015&END_WRTTIME=2025&CLS_ID=530090


,STATBL_ID,DTACYCLE_CD,WRTTIME_IDTFR_ID,GRP_ID,GRP_NM,CLS_ID,CLS_NM,ITM_ID,ITM_NM,DTA_VAL,UI_NM,GRP_FULLNM,CLS_FULLNM,ITM_FULLNM,WRTTIME_DESC
0,A_2024_00064,MM,201501,None,None,530088,덕양구,100001,가격,180642,천원,None,경기>경의권>고양시>덕양구,가격,2015년 1월
1,A_2024_00064,MM,201502,None,None,530088,덕양구,100001,가격,182660,천원,None,경기>경의권>고양시>덕양구,가격,2015년 2월
2,A_2024_00064,MM,201503,None,None,530088,덕양구,100001,가격,184698,천원,None,경기>경의권>고양시>덕양구,가격,2015년 3월
3,A_2024_00064,MM,201504,None,None,530088,덕양구,100001,가격,186972,천원,None,경기>경의권>고양시>덕양구,가격,2015년 4월
4,A_2024_00064,MM,201505,None,None,530088,덕양구,100001,가격,188802,천원,None,경기>경의권>고양시>덕양구,가격,2015년 5월
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,A_2024_00064,MM,202404,None,None,530090,일산서구,100001,가격,308509,천원,None,경기>경의권>고양시>일산서구,가격,2024년 4월
344,A_2024_00064,MM,202405,None,None,530090,일산서구,100001,가격,310231,천원,None,경기>경의권>고양시>일산서구,가격,2024년 5월
345,A_2024_00064,MM,202406,None,None,530090,일산서구,100001,가격,311138,천원,None,경기>경의권>고양시>일산서구,가격,2024년 6월
346,A_2024_00064,MM,202407,None,None,530090,일산서구,100001,가격,312961.62,천원,None,경기>경의권>고양시>일산서구,가격,2024년 7월


In [11]:
goyang_apt_tx = apt_indi.drop(['DTACYCLE_CD', 'STATBL_ID','GRP_ID','GRP_NM','CLS_ID','ITM_ID','UI_NM','GRP_FULLNM', 'CLS_FULLNM','ITM_FULLNM'], axis=1)
goyang_apt_tx.isnull().sum()

WRTTIME_IDTFR_ID    0
CLS_NM              0
ITM_NM              0
DTA_VAL             0
WRTTIME_DESC        0
dtype: int64

In [15]:
# 수치형식 변환
goyang_apt_tx['WRTTIME_IDTFR_ID'] = goyang_apt_tx['WRTTIME_IDTFR_ID'].astype(int)
goyang_apt_tx['DTA_VAL'] = goyang_apt_tx['DTA_VAL'].astype(float)
goyang_apt_tx['DTA_VAL'] = goyang_apt_tx['DTA_VAL']/10

In [16]:
goyang_apt_tx

,WRTTIME_IDTFR_ID,CLS_NM,ITM_NM,DTA_VAL,WRTTIME_DESC
0,201501,덕양구,가격,18064.200000,2015년 1월
1,201502,덕양구,가격,18266.000000,2015년 2월
2,201503,덕양구,가격,18469.800000,2015년 3월
3,201504,덕양구,가격,18697.200000,2015년 4월
4,201505,덕양구,가격,18880.200000,2015년 5월
...,...,...,...,...,...
343,202404,일산서구,가격,30850.900000,2024년 4월
344,202405,일산서구,가격,31023.100000,2024년 5월
345,202406,일산서구,가격,31113.800000,2024년 6월
346,202407,일산서구,가격,31296.162000,2024년 7월


In [17]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd

# 데이터 전처리
goyang_monthly_tx = goyang_apt_tx.copy()
goyang_monthly_tx['WRTTIME_IDTFR_ID'] = pd.to_datetime(goyang_monthly_tx['WRTTIME_IDTFR_ID'], format='%Y%m')
monthly_avg_tx = goyang_monthly_tx.groupby('WRTTIME_IDTFR_ID')['DTA_VAL'].mean().reset_index()

# 초기 데이터 필터링
filtered_data = monthly_avg_tx

# y축 최솟값과 최댓값 계산
y_min = filtered_data['DTA_VAL'].min()
y_min_with_margin = y_min - (y_min * 0.03)  # 3% 마진 추가
y_max = filtered_data['DTA_VAL'].max()
y_max_with_margin = y_max + (y_max * 0.03)  # 3% 마진 추가

# Plotly 그래프 생성
fig = px.line(filtered_data, 
              x='WRTTIME_IDTFR_ID', 
              y='DTA_VAL', 
              markers=False, 
              line_shape='spline',
              #labels={'WRTTIME_IDTFR_ID': '거래월', 'DTA_VAL': '평균매매가격(만원)'}
              )

# 선 그래프 색상 및 굵기 설정
fig.update_traces(line=dict(color='#007BFF', width=5))  # 밝고 비비드한 파란색

# y축 라벨 형식 수정
tickvals = list(range(int(y_min_with_margin // 10000 * 10000), int(y_max_with_margin // 10000 * 10000) + 1000, 10000))  # 리스트로 변환
ticktext = [f"{int(value // 10000)}억" for value in tickvals]  # 형식 수정

# 그래프 레이아웃 업데이트 (크기, 제목, 축 스타일, 배경색 등)
fig.update_layout(
    # title={
    #     'text': '고양시 아파트 월별 평균매매가격',
    #     'y': 0.95,
    #     'x': 0.5,
    #     'xanchor': 'center',
    #     'yanchor': 'top',
    #     'font': {'size': 28, 'family': 'Montserrat', 'color': '#2B3E50'}
    # },
    #template="plotly_white",  # 화이트 테마 적용
    xaxis=dict(
        showline=True,
        showticklabels=False,
        title=None,
        # title='거래월 (2022~2024년)',
        # tickfont=dict(size=16, family='Roboto', color='#3E3E3E'),  # 큰 글꼴 적용
        # titlefont=dict(size=20, family='Roboto', color='#2B3E3E'),
        # tickmode='linear',
        # dtick='M1',  # 매월 눈금 표시
        showgrid=False,  # 세로선
        # #gridcolor='rgba(211, 211, 211, 0.5)',  # 연한 회색 점선
        # gridwidth=1
    ),
    yaxis=dict(
        showline=True,
        showticklabels=True,
        title=None,
        tickfont=dict(size=25, family='Verdana', color='gray'),
        titlefont=dict(size=20, family='Roboto', color='#2B3E50'),
        tickmode='array',  # 수동으로 tick 설정
        tickvals=tickvals,  # y축 눈금 위치
        ticktext=ticktext,  # y축 눈금 라벨
        # showgrid=True,  # 가로선 표시
        # gridcolor='rgba(211, 211, 211, 0.5)',  # 연한 회색 점선
        # gridwidth=1,
        range=[y_min_with_margin, y_max_with_margin]  # y축의 최솟값과 최댓값에 마진 추가
    ),
    width=750,  # 그래프 너비
    height=500,  # 그래프 높이
    plot_bgcolor='white',  # 그래프 배경색을 깔끔한 흰색으로
    paper_bgcolor='white',   # 전체 배경색을 가벼운 회색
    xaxis_tickformat='%Y년',
    #xaxis_tickangle=45,  # x축 눈금 기울기
    margin=dict(l=40, r=40, t=80, b=60),  # 여백 조정
)

# y축 점선 추가
for y_value in range(int(y_min_with_margin//10000*10000), int(y_max_with_margin//10000*10000)+10000, 10000):
    fig.add_shape(type='line',
                  xref='paper', x0=0, x1=1,
                  yref='y', y0=y_value, y1=y_value,
                  line=dict(color="rgba(180,180,180,0.7)", width=1, dash="solid"))  # 점선과 굵기 설정

# x축 점선 추가
# for x_value in filtered_data['WRTTIME_IDTFR_ID']:
#     fig.add_shape(type='line',
#                   yref='paper', y0=0, y1=1,
#                   xref='x', x0=x_value, x1=x_value,
#                   line=dict(color="rgba(180,180,180,0.7)", width=1, dash="dot"))  # 점선과 굵기 설정

# 포인트 위에 텍스트 추가
# for i in range(len(filtered_data)):
#     deal_amount = filtered_data['DTA_VAL'].iloc[i]
#     billions = int(deal_amount // 10000)
#     ten_thousands = int(deal_amount % 10000)
# 
#     # 형식 수정: 5억2000만원
#     if billions > 0:
#         label = f'{billions}억{ten_thousands:04}만원'  # 4자리 수로 포맷
#     else:
#         label = f'{ten_thousands}만원'
#     
#     fig.add_annotation(
#         x=filtered_data['WRTTIME_IDTFR_ID'].iloc[i],
#         y=deal_amount,
#         text=label,
#         showarrow=False,
#         textangle=-25,  # 오른쪽으로 기울이기
#         xanchor='left',  # 텍스트의 시작점을 왼쪽으로 설정
#         yanchor='bottom',  # 텍스트의 시작점을 아래쪽으로 설정
#         font=dict(size=12, color="black"),  # 폰트 크기와 색상 설정
#         xshift=10  # 오른쪽으로 이동
#     )

# Dash 애플리케이션 초기화
app = dash.Dash(__name__)

# 레이아웃 정의
app.layout = html.Div([
    dcc.Graph(id='line-chart', figure=fig)
])

# 애플리케이션 실행
if __name__ == '__main__':
    app.run_server(debug=True, port="8051")
